In [1]:
!pip install peft -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 14.2 MB/s eta 0:00:00


In [2]:
!pip install transformers[torch] -q

In [3]:
import torch
import transformers
from peft import PeftModel, PeftConfig, PeftModel, LoraConfig, get_peft_model_state_dict
from transformers import GenerationConfig
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
trained_model = '/content/drive/MyDrive/МФТИ/nlp2/chatbot/generative/T5results'

config = PeftConfig.from_pretrained(trained_model)

In [6]:
model_name = 'google/flan-t5-base'

In [7]:
# Инициализация токенизатора
tokenizer = T5Tokenizer.from_pretrained(model_name)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# Инициализация токенизатора
#tokenizer = GPT2Tokenizer.from_pretrained(model_name)
#tokenizer.padding_side = "right"
#tokenizer.pad_token = tokenizer.eos_token

In [10]:
trained_model = T5ForConditionalGeneration.from_pretrained(
	config.base_model_name_or_path,
	device_map='auto'
	)

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [11]:
def generate_answer(query, model, tokenizer):
    # Токенизация запроса
    encoding = tokenizer(query, return_tensors="pt").to(model.device)

    # Конфигурация генерации текста
    generation_config = {
        "max_length": 250,
        "repetition_penalty": 1.3,
        "pad_token_id": tokenizer.eos_token_id,
        "eos_token_id": tokenizer.eos_token_id
    }

    # Генерация текста ответа
    outputs = model.generate(input_ids=encoding.input_ids, **generation_config)

    # Декодирование сгенерированного текста
    text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return text_output

In [17]:
query = "How do you get more girls?"
answer = generate_answer(query, trained_model, tokenizer)
print("Ответ модели:", answer)

Ответ модели: <pad> You should have a lot of girls.
